# <font color = '#FF003D'> ***Synchronous colorectal cancer-liver metastasis project***

**_______________________________________________________________________________________________________________________________________________________________________________________________________________**

# <font color = '#FF003D'> ***==== CODE 2: scVI integration & Myeloid cell re-clustering ====***

# Python library

In [ ]:
import os
import math
import warnings
import datetime

warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import scipy
import pandas as pd
import scanpy as sc
import scanpy.external as sce
from cycler import cycler
import openpyxl
import scvi

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

import rpy2

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import time
from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

In [ ]:
result_folder = '.../Analysis/Synchro/'
data_folder = '.../Analysis/Synchro/Data/'

sc.settings.verbosity = 4
warnings.filterwarnings('ignore')
sc.set_figure_params(dpi = 100, dpi_save = 1000, facecolor = 'white')

# R library

In [ ]:
! python -m rpy2.situation

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R 
library(Seurat)
library(SeuratDisk)
library(SeuratData)
library(SeuratWrappers)
library(SeuratObject)

library(openxlsx)
library(ggplot2)
library(ggraph)
library(ggrepel)

library(dplyr)
library(reticulate)
library(patchwork)

library(EnhancedVolcano)

# 1. **scVI integration**

In [ ]:
outfilename = os.path.join(data_folder, "Synchro_afterQC.h5ad")
adata = sc.read_h5ad(outfilename)

adata.layers["raw_counts"] = adata.X.copy()
adata

## 1.1. Normalization & HVG identification

In [ ]:
sc.pp.normalize_total(adata, target_sum = 1e4)
sc.pp.log1p(adata)
adata.raw = adata

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes = 4000,
                            flavor = 'seurat_v3', layer = "raw_counts",
                            batch_key = 'batch_id',
                            subset = False)

## 1.2. scVI integration

In [ ]:
adata_train = adata[:, adata.var.highly_variable].copy()
scvi.model.SCVI.setup_anndata(adata_train, layer = "raw_counts", batch_key = 'patient_ID')
vae = scvi.model.SCVI(adata_train, n_layers = 2, n_latent = 30, latent_distribution = "normal", gene_likelihood = "nb")
vae.train()

In [ ]:
adata.obsm["X_scVI"] = vae.get_latent_representation()
adata.obsm["denoised_RNA"] = vae.get_normalized_expression()

## 1.3. Neighbors & dimensionality reduction

In [ ]:
sc.pp.neighbors(adata, n_neighbors = 15, use_rep = "X_scVI", key_added = "scVI")
sc.tl.leiden(adata, resolution = 0.5, key_added = 'res_0.5', neighbors_key = "scVI")
sc.tl.paga(adata, groups = 'res_0.5', neighbors_key = "scVI")
sc.pl.paga(adata, frameon = True, edge_width_scale = 0.3)
sc.tl.umap(adata, neighbors_key = "scVI", init_pos = 'paga')

## 1.4. Clustering

In [ ]:
clustering_labels = []
for res in [0.5, 0.8, 1.0, 1.5, 1.8, 2.0, 2.5, 3.0]:
    clustering_labels.append("res_{}".format(res))
    if "res_{}".format(res) in adata.obs:
        print("res_{}".format(res) + " already exists... going on with next resolution.")
        continue
    sc.tl.leiden(adata, resolution = res, key_added = "res_{}".format(res), neighbors_key = "scVI")

## 1.5. Save the integrated object

In [ ]:
outfilename = os.path.join(data_folder, "Synchro_Integrated.h5ad")
print("Saving h5ad data to file {}".format(outfilename))
adata.write(outfilename)
print("Done!")

# 2. **Cell type identification**

In [ ]:
outfilename = os.path.join(data_folder, "Synchro_Integrated.h5ad")
adata = sc.read_h5ad(outfilename)
adata

## UMAP

In [ ]:
# Supplementary figure 1B
## Left UMAP
sc.pl.umap(adata, color = 'tissue_origin',
           save = 'Synchro total – tissue_origin.png'

## Right UMAP
sc.pl.umap(adata, color = 'patient_ID', frameon = True, palette = 'tab20', 
           save = 'Synchro total – patient_ID.png'

In [ ]:
# Supplementary figure 1C
sc.pl.umap(adata, color = 'res_1.5',
           legend_loc = 'on data', legend_fontsize = 14, legend_fontoutline = 2, legend_fontweight = 1, frameon = True,
           save = 'Synchro total – res_1.5.png'

## DotPlot

In [ ]:
# Supplementary figure 1D
order = ['31', '4', '24', '10', '27', '1', '21', '26', '7', '16', '8', '13', '6',
         '5',  '15', '11', '23', '12', '18', '28', '20', '3', '14', '17', '0', '9',  '22', 
         '2', '29', '19', '25', '32', '30']

signature = ['PTPRC', #CD45
             'CD14', 'FCGR3A', 'C5AR1', 'CD68', 'LYZ', 'VCAN', 'CD1C', 'CLEC10A', 'IL3RA', 'CLEC4C', 'CLEC9A', 'XCR1', #Myeloid cells
             'CD3D', 'CD3G', 'CD7', 'NKG7', 'NCAM1', 'KLRF1', #T cells & NK cells
             'MS4A1', 'CD19', 'JCHAIN', 'PRDM1', 'XBP1', #B cells and Plasma cells
             'CLDN5', 'ESAM', 'TUBB1', 'ITGA2B', #Basophils
             'TPSAB1', 'TPSB2', 'CTSG'] #Mast cells

sc.pl.dotplot(adata, signature, 'res_1.5', standard_scale = 'var', swap_axes = False,
              dendrogram = False, categories_order = order,
              dot_min = 0.1, dot_max = 1,
              save = 'Synchro total signature.png')

# 3. **Myeloid cells re-clustering**

In [ ]:
outfilename = os.path.join(data_folder, "Synchro_Integrated.h5ad")
adata = sc.read_h5ad(outfilename)
Myeloid = adata[adata.obs['res_1.5'].isin(['31', '4', '24', '10', '27', '1', '21', '26'])].copy()

obs = ['leiden', 'res_0', 'res_0.5', 'res_0.8', 'res_1.0', 'res_1.5', 'res_1.8', 'res_2.0', 'res_2.5', 'res_2.8', 'res_3.0']
for x in obs:
    del Myeloid.obs[x]

var = ['n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches']
for y in var:
    del Myeloid.var[y]

del Myeloid.uns
del Myeloid.obsm
del Myeloid.varm
del Myeloid.obsp
Myeloid

## 3.1. HVG identification

In [ ]:
sc.pp.highly_variable_genes(Myeloid, n_top_genes = 4000,
                            flavor = 'seurat_v3', layer = "raw_counts",
                            batch_key = 'batch_id',
                            subset = False)

## 3.2. scVI integration

In [ ]:
adata_train = Myeloid[:, Myeloid.var.highly_variable].copy()
scvi.model.SCVI.setup_anndata(adata_train, layer = "raw_counts", batch_key = 'patient_ID')
vae = scvi.model.SCVI(adata_train, n_layers = 2, n_latent = 30, latent_distribution = "normal", gene_likelihood = "nb")
vae.train()

In [ ]:
Myeloid.obsm["X_scVI"] = vae.get_latent_representation()
Myeloid.obsm["denoised_RNA"] = vae.get_normalized_expression()

## 3.3. Neighbors & dimensionality reduction

In [ ]:
sc.pp.neighbors(Myeloid, n_neighbors = 15, use_rep = "X_scVI", key_added = "scVI")
sc.tl.leiden(Myeloid, resolution = 0.5, key_added = 'res_0.5', neighbors_key = "scVI")
sc.tl.paga(Myeloid, groups = 'res_0.5', neighbors_key = "scVI")
sc.pl.paga(Myeloid, frameon = True, edge_width_scale = 0.3)
sc.tl.umap(Myeloid, neighbors_key = "scVI", init_pos = 'paga')

## 3.4. Clustering

In [ ]:
clustering_labels = []
for res in [0.5, 0.8, 1.0, 1.5, 1.8,]:
    clustering_labels.append("res_{}".format(res))
    if "res_{}".format(res) in Myeloid.obs:
        print("res_{}".format(res) + " already exists... going on with next resolution.")
        continue
    sc.tl.leiden(Myeloid, resolution = res, key_added = "res_{}".format(res), neighbors_key = "scVI")

## 3.5. Save the integrated object

In [ ]:
outfilename = os.path.join(data_folder, "Synchro_Reclustering_Myeloid.h5ad")
print("Saving h5ad data to file {}".format(outfilename))
Myeloid.write(outfilename)
print("Done!")